##one to many - cosine similarity 확인

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vector1 = np.random.randint(0,4,(1,5))
vector2 = np.random.randint(0,4,(3,5))
vector3 = np.concatenate((vector1,vector2),axis=0)
print(vector1,'\n')
print(vector2,'\n')
print(vector3,'\n')
similarity = cosine_similarity(vector1,vector3)
ttttt = similarity.argsort()
print(similarity)
print('sorted index : ',ttttt)

[[0 1 3 1 2]] 

[[2 3 2 0 0]
 [1 1 3 3 0]
 [0 2 0 3 1]] 

[[0 1 3 1 2]
 [2 3 2 0 0]
 [1 1 3 3 0]
 [0 2 0 3 1]] 

[[1.         0.56360186 0.75055535 0.48304589]]
sorted index :  [[3 1 2 0]]


In [ ]:
similarity

array([[0.71842121, 0.74466871, 0.76200076, 1.        ]])

In [ ]:
import json
import pandas as pd
import psycopg2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def recommend_by_theme(user_id, df, top):
  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']

  user_taste = calculate_taste(user_id)
  user_taste = user_taste.reshape(1,-1)

  print(user_taste)
  similarity = cosine_similarity(user_taste, df[tag])
  similarity = similarity[0]
  print(similarity.shape)
      
  print('similarity : ',similarity)
  
  result_index = similarity.argsort()
  result_index = result_index
  
  print("result_index : ",result_index)

  for i in range(-1,-6,-1):
    now_idx = result_index[i]
    sim = similarity[now_idx]
    print("similarity : ",sim,"df index : ",now_idx)

  top_N_result = result_index[-1:-(top+1):-1]
  print("top_N_result",top_N_result)
  
  result = df.iloc[top_N_result]
  
  return result

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

#추천메소드

##코스 데이터 중 len을 활용하여 day 범위에 해당하는 데이터만 불러오기

In [ ]:
def load_course(day,db):
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  load_course_sql= f'SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places FROM course_ WHERE length>={min_length} and length<={max_length}'
  cursor.execute(load_course_sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  df.columns = ['id','title','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','places']

  return df

###survey 기반 유저 취향 계산식 (평균)

In [ ]:
def calculate_taste(user_id,db):
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM tourist_attraction WHERE id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    print('place : ',place,"\tnow : ",place_theme)

  result = theme_score / len(survey_result)
  return result
  

In [ ]:
check_reuslt = calculate_taste(184)
print(check_reuslt)

[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
place :  32 	now :  (2, 3, 1, 1, 3, 1, 3, 0)
place :  19 	now :  (2, 3, 1, 2, 3, 0, 3, 0)
place :  13 	now :  (2, 3, 1, 3, 3, 1, 3, 0)
place :  6 	now :  (3, 2, 1, 1, 2, 1, 3, 1)
place :  33 	now :  (0, 0, 1, 1, 0, 3, 2, 0)
place :  8 	now :  (2, 3, 2, 3, 2, 2, 3, 0)
place :  10 	now :  (2, 2, 1, 1, 3, 1, 2, 0)
place :  25 	now :  (2, 3, 2, 3, 3, 2, 3, 0)
place :  41 	now :  (2, 3, 1, 1, 1, 2, 3, 0)
place :  72 	now :  (3, 3, 3, 0, 2, 0, 3, 0)
[2.  2.5 1.4 1.6 2.2 1.3 2.8 0.1]


###course data 로드메소드

In [ ]:
df

,id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places
0,2,[청주출발]연합패키지 노팁/노옵션 제주 3일_관광호텔 산방산유람선+ 화순곶자왈+서커...,1.714286,2.428571,1.285714,1.428571,2.571429,0.571429,2.285714,0.428571,3,"[19, 37, 10, 602, 60, 58, 29]"
1,6,[청주출발]◈감귤타르트증정/名品 효도◈고품격 노팁 노옵션-제주 로베로관광호텔 2박3...,2.500000,2.666667,1.833333,1.500000,2.000000,0.666667,2.833333,0.500000,8,"[37, 10, 65, 118, 58, 9]"
2,8,[광주출발]고품격 노팁 노옵션 관광호텔 2박3일 패키지 해저잠수함+잠수함+워터서커스...,2.000000,2.333333,1.500000,1.500000,1.666667,1.333333,2.333333,1.000000,10,"[37, 602, 78, 65, 118, 58]"
3,14,[코시롱한나들이] 신나는여름! 떠나요제주! 관광호텔 3일 산방산유람선+아쿠아플라넷+...,1.333333,2.000000,1.500000,1.333333,1.833333,1.166667,2.666667,1.000000,10,"[37, 78, 60, 202, 58, 29]"
4,15,[노팁노옵션 PERFECT 마라도투어] 제주 관광호텔 2박3일 카멜리아힐+마라도탐방...,1.875000,2.500000,1.875000,1.500000,2.375000,1.000000,2.375000,0.250000,3,"[138, 90, 10, 53, 602, 202, 58, 29]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2444,3804,제주도 여행,1.500000,1.916667,1.250000,1.083333,2.000000,1.000000,1.833333,0.583333,9,"[329, 18, 83, 6, 30, 3, 29, 1, 20, 10, 78, 59]"
2445,3805,똥강아지패밀리 신혼여행,2.538462,2.923077,1.384615,1.307692,1.692308,0.692308,2.307692,0.307692,4,"[15, 5, 597, 49, 20, 24, 9, 3, 42, 6, 439, 18,..."
2446,3807,제주도,2.454545,2.636364,1.454545,1.363636,1.818182,0.909091,2.090909,0.181818,1,"[59, 8, 5, 3, 439, 6, 22, 10, 24, 15, 9]"
2447,3809,제주도여행,1.444444,2.000000,1.333333,1.000000,1.666667,1.111111,2.444444,0.444444,9,"[18, 15, 87, 29, 9, 5, 179, 112, 61]"


In [ ]:
tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']

user_taste = calculate_taste(user_id)
user_taste.reshape(1,-1)

# print(user_taste)
# similarity = cosine_similarity(user_taste, df[tag])


[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
place :  32 	now :  (2, 3, 1, 1, 3, 1, 3, 0)
place :  19 	now :  (2, 3, 1, 2, 3, 0, 3, 0)
place :  13 	now :  (2, 3, 1, 3, 3, 1, 3, 0)
place :  6 	now :  (3, 2, 1, 1, 2, 1, 3, 1)
place :  33 	now :  (0, 0, 1, 1, 0, 3, 2, 0)
place :  8 	now :  (2, 3, 2, 3, 2, 2, 3, 0)
place :  10 	now :  (2, 2, 1, 1, 3, 1, 2, 0)
place :  25 	now :  (2, 3, 2, 3, 3, 2, 3, 0)
place :  41 	now :  (2, 3, 1, 1, 1, 2, 3, 0)
place :  72 	now :  (3, 3, 3, 0, 2, 0, 3, 0)
(8,)
<class 'numpy.ndarray'>


##최종 수정본
###db_load -> load_course-> calculate_taste -> recommend

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

def calculate_taste(user_id,db):
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM tourist_attraction WHERE id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    print('place : ',place,"\tnow : ",place_theme)

  result = theme_score / len(survey_result)
  return result
  
def load_course(day,db):
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  load_course_sql= f'SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places FROM course_ WHERE length>={min_length} and length<={max_length}'
  cursor.execute(load_course_sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  df.columns = ['id','title','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','places']

  return df
def recommend_by_theme(user_id, day):
  top = 5
  db = load_db()
  course = load_course(day,db)
  user_taste = calculate_taste(user_id,db)
  user_taste = user_taste.reshape(1,-1)

  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']

  print(user_taste)
  similarity = cosine_similarity(user_taste, course[tag])
  similarity = similarity[0]
  print(similarity.shape)
      
  print('similarity : ',similarity)
  
  result_index = similarity.argsort()
  result_index = result_index
  
  print("result_index : ",result_index)

  for i in range(-1,-6,-1):
    now_idx = result_index[i]
    sim = similarity[now_idx]
    print("similarity : ",sim,"df index : ",now_idx)

  top_N_result = result_index[-1:-(top+1):-1]
  print("top_N_result",top_N_result)
  
  result = course.iloc[top_N_result]
  
  return result

##테스트

In [ ]:
user_id = 184
day = 4

result = recommend_by_theme(user_id,day)

[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
place :  32 	now :  (2, 3, 1, 1, 3, 1, 3, 0)
place :  19 	now :  (2, 3, 1, 2, 3, 0, 3, 0)
place :  13 	now :  (2, 3, 1, 3, 3, 1, 3, 0)
place :  6 	now :  (3, 2, 1, 1, 2, 1, 3, 1)
place :  33 	now :  (0, 0, 1, 1, 0, 3, 2, 0)
place :  8 	now :  (2, 3, 2, 3, 2, 2, 3, 0)
place :  10 	now :  (2, 2, 1, 1, 3, 1, 2, 0)
place :  25 	now :  (2, 3, 2, 3, 3, 2, 3, 0)
place :  41 	now :  (2, 3, 1, 1, 1, 2, 3, 0)
place :  72 	now :  (3, 3, 3, 0, 2, 0, 3, 0)
[[2.  2.5 1.4 1.6 2.2 1.3 2.8 0.1]]
(1840,)
similarity :  [0.99016423 0.98707509 0.98717505 ... 0.97452429 0.98222014 0.99112965]
result_index :  [ 116 1291 1066 ... 1238 1257 1101]
similarity :  0.9993330202582142 df index :  1101
similarity :  0.998752727382239 df index :  1257
similarity :  0.9986198964474504 df index :  1238
similarity :  0.9983989925820225 df index :  115
similarity :  0.9983731353255855 df index :  1207
top_N_result [1101 1257 1238  115 1207]


In [ ]:
result

,id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places
1101,2263,제주도 2박3일,2.000000,2.375000,1.250000,1.500000,2.000000,1.125000,2.625000,0.125000,3,"[8, 8, 22, 179, 10, 60, 24, 18]"
1257,2566,혜리랑 제주도,2.125000,2.687500,1.625000,1.687500,2.250000,1.250000,2.750000,0.062500,5,"[25, 223, 707, 317, 379, 91, 4, 42, 179, 223, ..."
1238,2524,제주여행,1.700000,2.100000,1.100000,1.500000,1.800000,1.000000,2.500000,0.100000,3,"[80, 13, 30, 219, 76, 10, 38, 22, 174, 8]"
115,283,?사랑하는 사람과 함께하는 제주! 2박3일 여행?,1.666667,2.083333,1.333333,1.583333,2.000000,1.166667,2.500000,0.166667,3,"[80, 18, 13, 9, 17, 61, 55, 10, 22, 179, 31, 59]"
1207,2462,친구랑 2박3일 제주도 여행~♡,1.888889,2.111111,1.222222,1.555556,1.888889,1.111111,2.333333,0.000000,3,"[80, 38, 22, 10, 112, 8, 18, 188, 223]"
